In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import uuid
from os.path import expanduser, join
from re import findall, compile
from pandas import read_csv, merge, to_datetime, DataFrame
import subprocess

In [3]:
home = expanduser('~')
mypath = f'{home}/Downloads/BigData/'
deromalized_path = join(mypath, 'Datastax_denormalized/').replace('\\', '/')

## Connect to DataStax

In [4]:
cloud_config= {'secure_connect_bundle': mypath+'secure-connect-bigdataproject2022.zip'}

with open(mypath+'GeneratedToken.csv', 'r', encoding='utf-8') as file:
    try:
        content = file.readlines()        
    except FileNotFoundError:
        raise('File not found')

tokens = findall(r'"(.*?)"', content[1])
client_id = tokens[0]
client_secret = tokens[1]

In [5]:
keyspace = 'big_data'
auth_provider = PlainTextAuthProvider(client_id, client_secret)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect(keyspace)

In [9]:
def upload_data_to_cassandra(keyspace, table_name, filename, username, password):
    ds_bulk_url = 'C:/dsbulk-1.9.0/dsbulk-1.9.0/bin/dsbulk'
    filename_url = join(deromalized_path, filename).replace('\\', '/')
    bundle_url = join(mypath, 'secure-connect-bigdataproject2022.zip').replace('\\', '/')
    
    shell_comm = [ds_bulk_url, 'load', '-url', filename_url, '-k', keyspace, '-t', table_name, '-cl', 'ALL', 
                  '-b', bundle_url, '-u', username, '-p', password, '-header', 'true']
    
    print(' '.join(shell_comm))
    ret = subprocess.run(shell_comm, capture_output=True, shell=True)
    print(ret)
    return 

In [10]:
## maybe group by at first and store like it?
# function to clean out titles
def preprocess_movie_title(title):
    
    sep = ' ('
    title = title.split(sep, 1)[0]
    title = title.replace("'", "")
    
    comma_index = title.find(', The')
    a_index = title.find(', A')

    if comma_index > 0:
        return title[comma_index+2:] + ' ' + title[:comma_index]
    elif a_index > 0:
        return title[a_index+2:] + ' ' + title[:a_index]

    return title

In [11]:
# upload_data_to_cassandra(client_id, client_secret, 'bigdata', 'movies_by_title2', 'movies_by_title.csv')

## Q1

## Preprocess and denormalize data for Q1

In [25]:
movies = read_csv(mypath+'archive/movie.csv', names=["movieid", "title", "genres"], header=0)
print(movies.shape)
movies.head()

(27278, 3)


,movieid,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [30]:
# find how to keep the date part when parsing file u idiot
ratings = read_csv(mypath+'archive/rating.csv', usecols=['movieId', 'rating', 'timestamp'], header=0)
ratings = ratings.rename(columns={'movieId':'movieid'})
print(ratings.shape)
ratings.head()

(20000263, 3)


,movieId,rating,timestamp
0,2,3.5,2005-04-02 23:53:47
1,29,3.5,2005-04-02 23:31:16
2,32,3.5,2005-04-02 23:33:39
3,47,3.5,2005-04-02 23:32:07
4,50,3.5,2005-04-02 23:29:40


In [35]:
# create release date column
movies['release_date'] = movies['title'].str.extract(r"\((\d+)\)")

## clean titels and store back to column title
movies['title'] = movies['title'].apply(preprocess_movie_title)

movies.head()

,movieid,title,genres,release_date
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


### popular_movies_by_date

In [38]:
# create year column for partition key
ratings['year'] = ratings['timestamp'].apply(lambda x: int(x[:4]))

          movieid  rating            timestamp  year
0               2     3.5  2005-04-02 23:53:47  2005
1              29     3.5  2005-04-02 23:31:16  2005
2              32     3.5  2005-04-02 23:33:39  2005
3              47     3.5  2005-04-02 23:32:07  2005
4              50     3.5  2005-04-02 23:29:40  2005
...           ...     ...                  ...   ...
20000258    68954     4.5  2009-11-13 15:42:00  2009
20000259    69526     4.5  2009-12-03 18:31:48  2009
20000260    69644     3.0  2009-12-07 18:10:57  2009
20000261    70286     5.0  2009-11-13 15:42:24  2009
20000262    71619     2.5  2009-10-17 20:25:36  2009

[20000263 rows x 4 columns]


In [41]:
# left join with ratings
popular_movies_by_date = merge(left=movies.loc[:, 'movieid':'genres'], right=ratings, left_on='movieid', right_on='movieid')
popular_movies_by_date[:10]

,movieid,title,genres,rating,timestamp,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,4.0,1999-12-11 13:36:47,1999
1,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,5.0,1997-03-13 17:50:52,1997
2,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,4.0,1996-06-05 13:37:51,1996
3,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,4.0,1999-11-25 02:44:47,1999
4,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,4.5,2009-01-02 01:13:41,2009
5,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,4.0,1997-03-22 20:48:38,1997
6,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,4.0,1996-11-27 08:21:31,1996
7,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,4.5,2008-10-29 20:13:59,2008
8,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,3.0,2001-05-27 13:22:52,2001
9,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,5.0,1997-02-05 21:03:48,1997


### Insert data to table popular_movies_by_date

In [42]:
popular_movies_by_date[0::10000].to_csv(mypath+'Datastax_denormalized/popular_movies_by_date.csv', index=0)

In [45]:
upload_data_to_cassandra(keyspace, 'popular_movies_by_date', 'popular_movies_by_date.csv', client_id, client_secret)

C:/dsbulk-1.9.0/dsbulk-1.9.0/bin/dsbulk load -url C:/Users/giwrg/Downloads/BigData/Datastax_denormalized/popular_movies_by_date.csv -k big_data -t popular_movies_by_date -cl ALL -b C:/Users/giwrg/Downloads/BigData/secure-connect-bigdataproject2022.zip -u ZeYCAzFQsZzBoJQgHJWSitoc -p 8IZ_YFm-E,4g28Q.kw.JleZ5h,BdjYmFTUMZkKk2n5r8k7N0LtGILx3g44yeWo1yr7N,XTLZpu0,DwUHpBcUb8aziYEckRalTgh8.WnbJOuqrwpMcrl..As90BZYzJDj -header true
CompletedProcess(args=['C:/dsbulk-1.9.0/dsbulk-1.9.0/bin/dsbulk', 'load', '-url', 'C:/Users/giwrg/Downloads/BigData/Datastax_denormalized/popular_movies_by_date.csv', '-k', 'big_data', '-t', 'popular_movies_by_date', '-cl', 'ALL', '-b', 'C:/Users/giwrg/Downloads/BigData/secure-connect-bigdataproject2022.zip', '-u', 'ZeYCAzFQsZzBoJQgHJWSitoc', '-p', '8IZ_YFm-E,4g28Q.kw.JleZ5h,BdjYmFTUMZkKk2n5r8k7N0LtGILx3g44yeWo1yr7N,XTLZpu0,DwUHpBcUb8aziYEckRalTgh8.WnbJOuqrwpMcrl..As90BZYzJDj', '-header', 'true'], returncode=0, stdout=b'', stderr=b'Username and password provided but au

### Retrieve data from popular_movies_by_date

In [51]:
def select_from_popular_movies_by_date(start_date, end_date):
    Q1 = f"SELECT * FROM big_data.popular_movies_by_date WHERE year=? and timestamp >= ? and timestamp <= ?;"
    prepared = session.prepare(Q1)
    sd = to_datetime(start_date)
    ed = to_datetime(end_date)
    if sd.year == ed.year:
        rows = session.execute(prepared, (sd.year, sd, ed))
        for row in rows:
            print(f"Title: {row.title} Rating: {row.rating:.1f}")

In [52]:
select_from_popular_movies_by_date('2013-01-01', '2013-02-01')

Title: The Dark Knight Rating: 5.0
Title: Beasts of the Southern Wild Rating: 3.0
Title: Me and You and Everyone We Know Rating: 4.5
Title: Fight Club Rating: 3.5
Title: Hannibal Rising Rating: 4.0
Title: Vegas Vacation Rating: 3.0
Title: A Nightmare on Elm Street Rating: 4.0
Title: The Best Exotic Marigold Hotel Rating: 3.5


## Q2

### Preprocess and denormalize data for Q2

# Q3

## Model data for Q3

In [59]:
# split genres per row
b = DataFrame(movies['genres'].str.split('|').tolist(), index=movies['movieid']).stack()
b = b.reset_index()[[0, 'movieid']] 
b.columns = ['genre', 'movieid'] 

# left join the genres with movies
movies_by_genre = merge(left=b, right=movies.drop(columns='genres', axis=1), left_on='movieid', right_on='movieid')
# calc movie avg
avgs = ratings.groupby(by='movieid')['rating'].mean().reset_index().rename(columns={'rating':'avg_rating'}).round(decimals=1)

# join movies and ratins
movies_by_genre = merge(left=movies_by_genre, right=avgs, left_on='movieid', right_on='movieid')

# remove trash rows
movies_by_genre = movies_by_genre[movies_by_genre['release_date'].str.len() == 4]
movies_by_genre = movies_by_genre[movies_by_genre['genre'] != '(no genres listed)']

movies_by_genre

,genre,movieid,title,release_date,avg_rating
0,Adventure,1,Toy Story,1995,3.9
1,Animation,1,Toy Story,1995,3.9
2,Children,1,Toy Story,1995,3.9
3,Comedy,1,Toy Story,1995,3.9
4,Fantasy,1,Toy Story,1995,3.9
...,...,...,...,...,...
53414,Adventure,131258,The Pirates,2014,2.5
53415,(no genres listed),131260,Rentun Ruusu,2001,3.0
53416,Adventure,131262,Innocence,2014,4.0
53417,Fantasy,131262,Innocence,2014,4.0


### Store file and Insert to movies_by_genre

In [ ]:
# store to csv
movies_by_genre.to_csv(deromalized_path + 'movies_by_genre.csv', index=0)
upload_data_to_cassandra(keyspace, 'movies_by_genre', 'movies_by_genre.csv', client_id, client_secret)

### Retrieve data from movies_by_genre

In [63]:
def select_from_movies_by_genre(genre, limit=10):
    Q3 = f"SELECT title, avg_rating, release_date FROM movies_by_genre WHERE genre='{genre}' ORDER BY release_date ASC LIMIT {limit};"
    rows = session.execute(Q3)
    for row in rows:
        print(f"Title: {row.title} Rating: {row.avg_rating} Release Date: {row.release_date}")
    

In [64]:
# insert_into_movies_by_genre(movies_ratings, 1)
select_from_movies_by_genre('Adventure')

Title: A Trip to the Moon Rating: 3.7 Release Date: 1902
Title: The Last Days of Pompeii Rating: 2.5 Release Date: 1913
Title: Judith of Bethulia Rating: 3.3 Release Date: 1914
Title: Cabiria Rating: 3.1 Release Date: 1914
Title: Vampires, Les Rating: 3.4 Release Date: 1915
Title: Judex Rating: 3.5 Release Date: 1916
Title: 20,000 Leagues Under the Sea Rating: 3.3 Release Date: 1916
Title: Male and Female Rating: 2.9 Release Date: 1919
Title: The Spiders Part 1: The Golden Lake Rating: 2.8 Release Date: 1919
Title: The Last of the Mohicans Rating: 3.4 Release Date: 1920


## Q4

In [114]:
tags = read_csv(mypath+'archive/tag.csv', usecols=['movieId', 'tag'])
tags = tags.rename(columns={'movieId':'movieid'})
tags['tag'].replace(r'[^\w\s]','',regex=True, inplace = True)
tags = tags[tags['tag'].str.len() > 1]

tags.head()

AttributeError: 'Series' object has no attribute 'len'

In [91]:
movies_by_title = merge(left=tags, right=movies.loc[:, 'movieid':'genres'], left_on='movieid', right_on='movieid')
print(movies_by_title.shape)
movies_by_title.head()

(465564, 4)


,movieid,tag,title,genres
0,4141,Mark Waters,Head Over Heels,Comedy|Romance
1,4141,naive,Head Over Heels,Comedy|Romance
2,4141,Mark Waters,Head Over Heels,Comedy|Romance
3,4141,Freddie Prinze Jr,Head Over Heels,Comedy|Romance
4,208,dark hero,Waterworld,Action|Adventure|Sci-Fi


In [104]:
movies_by_title = movies_by_title.groupby(by=['movieid', 'tag'])['tag'].count().reset_index(name='tag_count')
movies_by_title = merge(left=movies_by_title, right=movies.loc[:, 'movieid':'genres'], left_on='movieid', right_on='movieid')
movie_info_by_title = merge(left=movies_by_title, right=avgs, left_on='movieid', right_on='movieid')
print(movie_info_by_title.shape)
movie_info_by_title.head()

,movieid,tag,tag_count,title,genres,avg_rating
0,1,2009 reissue in Stereoscopic 3D,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,3.9
1,1,3D,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,3.9
2,1,55 movies every kid should seeEntertainment We...,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,3.9
3,1,Animation,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,3.9
4,1,BDVideo,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,3.9


### Store file and Insert to movies_by_title

In [109]:
# store to csv
movie_info_by_title.to_csv(deromalized_path + 'movie_info_by_title.csv', index=0)
upload_data_to_cassandra(keyspace, 'movie_info_by_title', 'movie_info_by_title.csv', client_id, client_secret)

C:/dsbulk-1.9.0/dsbulk-1.9.0/bin/dsbulk load -url C:/Users/giwrg/Downloads/BigData/Datastax_denormalized/movies_by_title.csv -k big_data -t movies_by_title -cl ALL -b C:/Users/giwrg/Downloads/BigData/secure-connect-bigdataproject2022.zip -u ZeYCAzFQsZzBoJQgHJWSitoc -p 8IZ_YFm-E,4g28Q.kw.JleZ5h,BdjYmFTUMZkKk2n5r8k7N0LtGILx3g44yeWo1yr7N,XTLZpu0,DwUHpBcUb8aziYEckRalTgh8.WnbJOuqrwpMcrl..As90BZYzJDj -header true
CompletedProcess(args=['C:/dsbulk-1.9.0/dsbulk-1.9.0/bin/dsbulk', 'load', '-url', 'C:/Users/giwrg/Downloads/BigData/Datastax_denormalized/movies_by_title.csv', '-k', 'big_data', '-t', 'movies_by_title', '-cl', 'ALL', '-b', 'C:/Users/giwrg/Downloads/BigData/secure-connect-bigdataproject2022.zip', '-u', 'ZeYCAzFQsZzBoJQgHJWSitoc', '-p', '8IZ_YFm-E,4g28Q.kw.JleZ5h,BdjYmFTUMZkKk2n5r8k7N0LtGILx3g44yeWo1yr7N,XTLZpu0,DwUHpBcUb8aziYEckRalTgh8.WnbJOuqrwpMcrl..As90BZYzJDj', '-header', 'true'], returncode=0, stdout=b'', stderr=b'Username and password provided but auth provider not specified, i

In [131]:
def select_from_movies_by_title(session, title):
    Q4 = f"SELECT title, genres, tag, avg_rating AS avg FROM movies_by_title WHERE title='{title}' ORDER BY tag_count LIMIT 5;"
    rows = session.execute(Q4)
    print(f"Title: {rows[0].title} \nRating: {rows[0].avg} \nGenres: {rows[0].genres}")
    print(f"Most popular tags: ")
    for row in rows:
        print(row.tag)

In [132]:
select_from_movies_by_title(session, 'Jumanji')

Title: Jumanji 
Rating: 3.200000047683716 
Genres: Adventure|Children|Fantasy
Most popular tags: 
time travel
time
see alsoZathura
scary
not for kids


## Q5

## Data Denormalization for Q5

In [31]:
mrt = merge(left=tags.drop_duplicates(), right=movies.loc[:, 'movieId':'title'], left_on='movieId', right_on='movieId')
movies_by_tag = merge(left=mrt, right=avgs, left_on='movieId', right_on='movieId')

In [34]:
movies_by_tag.to_csv(mypath+'Datastax_denormalized\\movies_by_tag.csv', index=0)

In [93]:
def insert_into_movies_by_tag(df, n):
    # insert data for Q4
    sep = '('
    for index, row in df[0::n].iterrows():
        #  tag | movieid | rating | id | title
        tag = row.tag.replace("'", "").split(sep, 1)[0]
        movie_id = row.movieId
        r = row.rating
        idd = uuid.uuid4()
        title = row.title[:-6].replace("'", "").split(sep, 1)[0]
        
        q = f"INSERT INTO movies_by_tag(tag, movieid, rating, id, title)VALUES('{tag}', {movie_id}, {r}, {idd}, '{title}')"
        session.execute(q)

In [103]:
def select_from_movies_by_tag(tag):
    Q5 = f"SELECT tag, title, avg(rating) AS avg FROM movies_by_tag WHERE tag='{tag}' GROUP BY movieid;"
    rows = session.execute(Q5)
    for row in rows:
        print(f"Tag: {row.tag} Title: {row.title} Rating: {row.avg:.1f}")
    

In [104]:
# insert_into_movies_by_tag(mrt, 1)
select_from_movies_by_tag('comedy')

Tag: comedy Title: Cars 1 Rating: 4.2
Tag: comedy Title: Lost highway Rating: 4.8
Tag: comedy Title: Robin Hood: Men in Tights  Rating: 3.0
Tag: comedy Title: Welcome to the Dollhouse  Rating: 4.5
Tag: comedy Title: Home Alone  Rating: 3.0
Tag: comedy Title: Raiders of the Lost Ark  Rating: 3.8
Tag: comedy Title: Blues Brothers, The  Rating: 4.5
Tag: comedy Title: Life Is Beautiful  Rating: 3.5
Tag: comedy Title: Office Space  Rating: 4.2
Tag: comedy Title: Animal House  Rating: 5.0
Tag: comedy Title: Bridget Joness Diary  Rating: 3.0
Tag: comedy Title: Shrek  Rating: 4.2
Tag: comedy Title: Adaptation  Rating: 4.0
